In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [2]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

In [3]:
# Save (commit) the changes
conn.commit()

In [4]:
last_sale = (None, 4000, 5, 22, 619000)
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)
# Remember to commit the changes
conn.commit()

In [5]:
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

In [10]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists
data = (genfromtxt('./assets/datasets/housing-data.csv', dtype='i8',
                    delimiter=',', skip_header=1)).tolist()

# append a None value to beginning of each sub-list
for d in data:
    d.insert(0, None)

In [11]:
data[0:3]

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000]]

In [12]:
# loop through data, running an INSERT on each record (i.e. sublist)
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

Check: How do you delete data?

In [ ]:
# results = c.execute('delete from data where field1=2')

In [18]:
# Similar syntax as before
results = c.execute("SELECT * FROM houses WHERE bdrms = 4")

# Here results is a cursor object - use fetchall() to extract a list
results.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

Pandas connector

In [19]:
import pandas as pd
from pandas.io import sql

Writing data into a database

In [20]:
data = pd.read_csv('./assets/datasets/housing-data.csv', low_memory=False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


Data is moved to the database through the to_sql command, similar to the to_csv command.

 if we want to write a custom schema for the new table - if_exists, what to do if the table already exists. We can overwrite it, add to it, or fail



In [26]:
data.to_sql('houses_pandas',
            con=conn,
            if_exists='replace',
            index=False)

Reading data from a database
If we already have data in our database we can use Pandas to query it

In [27]:
sql.read_sql('select * from houses_pandas limit 10', con=conn)

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900
5,1985,4,61,299900
6,1534,3,12,314900
7,1427,3,57,198999
8,1380,3,14,212000
9,1494,3,15,242500


In [37]:
sql.read_sql('select bdrms, avg(price) from houses_pandas where bdrms=1 limit 10', con=conn)

,bdrms,avg(price)
0,1,169900.0


In [38]:
sql.read_sql('select bdrms, avg(price) from houses_pandas where bdrms=2', con=conn)

,bdrms,avg(price)
0,2,280866.666667


In [42]:
sql.read_sql('select bdrms, count(bdrms) from houses_pandas group by bdrms limit 10', con=conn)

,bdrms,count(bdrms)
0,1,1
1,2,6
2,3,25
3,4,14
4,5,1


In [46]:
sql.read_sql('select sqft, count(sqft) from houses_pandas group by sqft limit 10', con=conn)

,sqft,count(sqft)
0,852,1
1,1000,1
2,1100,1
3,1200,1
4,1203,1
5,1236,1
6,1239,1
7,1268,1
8,1320,1
9,1380,1


In [31]:
results = c.execute("SELECT sqft, bdrms FROM houses_pandas")

In [33]:
# results.fetchall()

Questions:

What's the average price per room for 1 bedroom apartments?
What's the average price per room for 2 bedrooms apartments?
What's the most frequent apartment size (in terms of bedrooms)?
How many are there of that apartment kind?
What fraction of the total number are of that kind?
How old is the oldest 3 bedrooms apartment?
How old is the youngest apartment?
What's the average age for the whole dataset?
What's the average age for each bedroom size?
Try to answer all these in SQL.

If you finish, try completing the first sections of SQL zoo.